In [14]:
import sys, os
os.chdir('/Users/nick/Documents/school/research/alpr-unconstrained-modified')
import keras
import cv2

from src.keras_utils import load_model
from glob import glob
from os.path import splitext, basename
from src.utils import nms
from src.keras_utils import load_model, detect_lp
from src.label import Shape, writeShapes, dknet_label_conversion
import darknet.python.darknet as dn
from darknet.python.darknet import detect

import pandas as pd
import matplotlib.pyplot as plt
from tqdm import trange

In [15]:
det_repo = '../EfficientLPR'
car_predictions_path = det_repo + '/predictions_phi0_nzvd.csv'
car_predictions = pd.read_csv(car_predictions_path)

lp_threshold = 0.5
ocr_threshold = 0.4

wpod_net_path = "data/lp-detector/wpod-net_update1.h5"
ocr_weights = 'data/ocr/ocr-net.weights'
ocr_netcfg  = 'data/ocr/ocr-net.cfg'
ocr_dataset = 'data/ocr/ocr-net.data'

if not os.path.exists('tmp'):
    os.mkdir('tmp')

wpod_net = load_model(wpod_net_path)

ocr_net  = dn.load_net(ocr_netcfg.encode('utf-8'), ocr_weights.encode('utf-8'), 0)
ocr_meta = dn.load_meta(ocr_dataset.encode('utf-8'))

predicted_lps = []

for i in trange(len(car_predictions)):
    sample = car_predictions.iloc[i]
    fname = sample[['file']].values.item()

    image_name = fname.split('/')[-1]

    # read image
    im_path = det_repo + '/' + fname
    img = cv2.imread(im_path)
    src_image = img.copy()

    # get bbox
    x1, y1, x2, y2 = [int(x) for x in sample[['x1', 'y1', 'x2', 'y2']].values]

    # crop image to bbox
    img = img[y1:y2, x1:x2]

    im_shape = img.shape[:2]
    min_dim, max_dim = min(im_shape), max(im_shape)

    ratio = max_dim/min_dim
    side  = int(ratio*288)
    bound_dim = min(side + (side%(2**4)),608)


    lps, rectified_images, _ = detect_lp(wpod_net,img/255,bound_dim,2**4,(240,80),lp_threshold)

    predicted_lp = ''

    if len(lps):
        best_idx = 0 # index of highest score
        lp = lps[0]
        rectified_image = rectified_images[0]

        # save rectified image so we can pass to darkflow
        out_path = 'tmp/' + image_name
        cv2.imwrite(out_path, rectified_image*255)

        # read license plate
        results,(width,height) = detect(ocr_net, ocr_meta, out_path.encode('utf-8'), thresh=ocr_threshold, nms=None)

        if len(results):
            labels = dknet_label_conversion(results,width,height)
            labels = nms(labels,.45)
            # get characters
            labels.sort(key=lambda x: x.tl()[0])
            letters = [chr(l.cl()) for l in labels]
            predicted_lp = "".join(letters)
        os.remove(out_path)
    
    predicted_lps.append(predicted_lp)

car_predictions = car_predictions.assign(lp=pd.Series(predicted_lps))

out_path = car_predictions_path[:-3] + '.full.csv'
car_predictions.to_csv(out_path)

100%|██████████| 100/100 [00:34<00:00,  2.87it/s]
